In [17]:
import pandas as pd
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


import categorical_embedder as ce
import numpy as np

In [18]:
df = pd.read_csv('../Machine_Learning/df2.csv')

In [19]:
df

,y_true,symptom,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,...,DE,De,dE,de,SERIOUS Y,SERIOUS N,manufacturer_JANSSEN,manufacturer_MODERNA,manufacturer_NOVAVAX,manufacturer_PFIZER\BIONTECH
0,1,Angioedema,23,95,161,304,308,287,191,105,...,2878.0,943022.0,2351.0,1530710.0,564.0,2314.0,0,0,0,1
1,1,Hypoaesthesia,39,732,2295,4248,4426,3566,1960,862,...,26529.0,919371.0,27845.0,1505216.0,7405.0,19126.0,0,0,0,1
2,1,Erythema multiforme,20,12,31,28,41,30,34,30,...,733.0,945167.0,2228.0,1530833.0,131.0,602.0,0,0,0,1
3,1,Insomnia,33,205,732,1345,1345,1253,1012,581,...,11215.0,934685.0,14609.0,1518452.0,2674.0,8541.0,0,0,0,1
4,1,Myalgia,106,935,4266,7133,6992,6599,4653,2335,...,52182.0,893718.0,75910.0,1457151.0,12832.0,39351.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,0,Increased insulin requirement,0,0,1,1,0,0,1,0,...,3.0,98389.0,33.0,2380536.0,2.0,1.0,1,0,0,0
1121,0,Blood lactate dehydrogenase decreased,0,0,0,0,1,0,0,0,...,7.0,945893.0,17.0,1533044.0,4.0,3.0,0,0,0,1
1122,0,Breast cellulitis,0,0,0,0,2,0,0,2,...,4.0,537275.0,16.0,1941666.0,1.0,3.0,0,1,0,0
1123,0,Vascular graft thrombosis,0,0,0,0,0,2,0,1,...,3.0,537276.0,10.0,1941672.0,3.0,0.0,0,1,0,0


In [20]:
X = df[['symptom']]
y = df['y_true']
print(X.shape)
print(y.shape)

(1125, 1)
(1125,)


In [21]:
y.values

array([1, 1, 1, ..., 0, 0, 0])

In [22]:
embedding_info = ce.get_embedding_info(X)
X_encoded, encoders = ce.get_label_encoded_data(X)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y)

# Get embeddings
embeddings = ce.get_embeddings(X_train, y_train, categorical_embedding_info=embedding_info,
                               is_classification=True, epochs=100, batch_size=256)


Epoch 1/100
3/3 [==============================] - 1s 62ms/step - loss: 0.6552 - accuracy: 0.8887 - val_loss: 0.5828 - val_accuracy: 0.8817
Epoch 2/100
3/3 [==============================] - 0s 20ms/step - loss: 0.5401 - accuracy: 0.8887 - val_loss: 0.4682 - val_accuracy: 0.8817
Epoch 3/100
3/3 [==============================] - 0s 15ms/step - loss: 0.4156 - accuracy: 0.8887 - val_loss: 0.3743 - val_accuracy: 0.8817
Epoch 4/100
3/3 [==============================] - 0s 18ms/step - loss: 0.3285 - accuracy: 0.8887 - val_loss: 0.3498 - val_accuracy: 0.8817
Epoch 5/100
3/3 [==============================] - 0s 20ms/step - loss: 0.3053 - accuracy: 0.8887 - val_loss: 0.3600 - val_accuracy: 0.8817
Epoch 6/100
3/3 [==============================] - 0s 17ms/step - loss: 0.2885 - accuracy: 0.8887 - val_loss: 0.3436 - val_accuracy: 0.8817
Epoch 7/100
3/3 [==============================] - 0s 18ms/step - loss: 0.2438 - accuracy: 0.8887 - val_loss: 0.3136 - val_accuracy: 0.8817
Epoch 8/100
3/3 [===

In [23]:
df_embedded = ce.fit_transform(df, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
df_embedded.head()

,y_true,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,...,symptom_embedding_40,symptom_embedding_41,symptom_embedding_42,symptom_embedding_43,symptom_embedding_44,symptom_embedding_45,symptom_embedding_46,symptom_embedding_47,symptom_embedding_48,symptom_embedding_49
0,1,23,95,161,304,308,287,191,105,45,...,0.027804,0.023666,0.003403,-0.010580,-0.038636,-0.033641,-0.014515,-0.044567,0.004077,0.024810
1,1,39,732,2295,4248,4426,3566,1960,862,241,...,-0.053546,-0.071147,-0.063355,-0.023203,0.005357,-0.005699,-0.055387,0.047468,0.067334,0.048406
2,1,20,12,31,28,41,30,34,30,20,...,-0.021373,-0.042889,-0.036618,0.022410,0.019555,-0.025536,-0.027378,-0.012780,0.044856,0.032292
3,1,33,205,732,1345,1345,1253,1012,581,180,...,0.017694,-0.016068,0.042424,-0.024262,-0.010284,-0.035142,0.038593,0.047516,-0.000315,-0.035767
4,1,106,935,4266,7133,6992,6599,4653,2335,583,...,-0.033978,-0.075673,-0.051500,-0.045333,0.097312,0.039915,-0.019549,0.042446,0.066608,0.039786


In [24]:
num_attribs = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99', '100-109',
               '110-119', 'NUMDAYS_0-1', 'NUMDAYS_2-3', 'NUMDAYS_4-5', 'NUMDAYS_6-7', 'NUMDAYS_8-9', 'NUMDAYS_10-11',
               'NUMDAYS_12-13', 'NUMDAYS_14+', 'DIED', 'L_THREAT', 'HOSPITAL', 'X_STAY', 'DISABLE', 'BIRTH_DEFECT',
               'DE', 'De', 'dE', 'de', 'SERIOUS Y', 'SERIOUS N', 'SEX_M', 'SEX_F', 'SEX_U']

# Separate features and target
X = df_embedded.drop(columns=['y_true'])
y_true = df_embedded['y_true']

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('min_max_scaler', MinMaxScaler())
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs)
], remainder='passthrough')

# Apply transformation
X_transformed = full_pipeline.fit_transform(X)

# Combine the transformed features with the target
df_transformed = pd.DataFrame(X_transformed, columns=X.columns)

In [38]:
df_transformed.columns

Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80-89', '90-99', '100-109', '110-119', 'NUMDAYS_0-1', 'NUMDAYS_2-3',
       'NUMDAYS_4-5', 'NUMDAYS_6-7', 'NUMDAYS_8-9', 'NUMDAYS_10-11',
       'NUMDAYS_12-13', 'NUMDAYS_14+', 'SEX_M', 'SEX_F', 'SEX_U', 'DIED',
       'L_THREAT', 'HOSPITAL', 'X_STAY', 'DISABLE', 'BIRTH_DEFECT', 'DE', 'De',
       'dE', 'de', 'SERIOUS Y', 'SERIOUS N', 'manufacturer_JANSSEN',
       'manufacturer_MODERNA', 'manufacturer_NOVAVAX',
       'manufacturer_PFIZER\BIONTECH', 'symptom_embedding_0',
       'symptom_embedding_1', 'symptom_embedding_2', 'symptom_embedding_3',
       'symptom_embedding_4', 'symptom_embedding_5', 'symptom_embedding_6',
       'symptom_embedding_7', 'symptom_embedding_8', 'symptom_embedding_9',
       'symptom_embedding_10', 'symptom_embedding_11', 'symptom_embedding_12',
       'symptom_embedding_13', 'symptom_embedding_14', 'symptom_embedding_15',
       'symptom_embedding_16', 'symptom_embedding_1

In [36]:
X_train, X_test, y_train, y_test = train_test_split(df_transformed, y_true, test_size=0.2, random_state=42)

X_train.to_csv('../Machine_Learning/X_train.csv', index=False)
X_test.to_csv('../Machine_Learning/X_test.csv', index=False)
y_train.to_csv('../Machine_Learning/y_train.csv', index=False)
y_test.to_csv('../Machine_Learning/y_test.csv', index=False)

# Generate splits with Moderna = test

In [41]:
df_transformed['y_true'] = y_true

In [43]:
df_transformed

,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99,...,symptom_embedding_41,symptom_embedding_42,symptom_embedding_43,symptom_embedding_44,symptom_embedding_45,symptom_embedding_46,symptom_embedding_47,symptom_embedding_48,symptom_embedding_49,y_true
0,0.020193,0.016728,0.014843,0.017429,0.017247,0.016329,0.012564,0.012005,0.013682,0.006903,...,0.023666,0.003403,-0.010580,-0.038636,-0.033641,-0.014515,-0.044567,0.004077,0.024810,1
1,0.034241,0.128896,0.211579,0.243550,0.247844,0.202890,0.128930,0.098559,0.073275,0.034517,...,-0.071147,-0.063355,-0.023203,0.005357,-0.005699,-0.055387,0.047468,0.067334,0.048406,1
2,0.017559,0.002113,0.002858,0.001605,0.002296,0.001707,0.002237,0.003430,0.006081,0.003945,...,-0.042889,-0.036618,0.022410,0.019555,-0.025536,-0.027378,-0.012780,0.044856,0.032292,1
3,0.028973,0.036098,0.067484,0.077113,0.075316,0.071290,0.066570,0.066430,0.054728,0.023669,...,-0.016068,0.042424,-0.024262,-0.010284,-0.035142,0.038593,0.047516,-0.000315,-0.035767,1
4,0.093064,0.164642,0.393288,0.408955,0.391533,0.375455,0.306078,0.266979,0.177258,0.074951,...,-0.075673,-0.051500,-0.045333,0.097312,0.039915,-0.019549,0.042446,0.066608,0.039786,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,0.000000,0.000000,0.000092,0.000057,0.000000,0.000000,0.000066,0.000000,0.000000,0.000000,...,0.041476,-0.047308,0.065455,-0.010445,0.056432,-0.000285,-0.030303,-0.051557,-0.050124,0
1121,0.000000,0.000000,0.000000,0.000000,0.000056,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.015479,-0.051956,-0.038010,-0.055746,-0.053685,0.059090,-0.029842,0.027922,-0.042372,0
1122,0.000000,0.000000,0.000000,0.000000,0.000112,0.000000,0.000000,0.000229,0.000000,0.000000,...,-0.031900,0.008725,0.036383,-0.007435,0.030956,-0.030577,0.022444,-0.003479,0.038715,0
1123,0.000000,0.000000,0.000000,0.000000,0.000000,0.000114,0.000000,0.000114,0.000000,0.000000,...,-0.025027,-0.017652,-0.038262,-0.032255,-0.015502,0.032897,0.017856,-0.038065,0.030324,0


In [78]:
X_test = df_transformed[(df_transformed['manufacturer_MODERNA'] == 1)]
y_test = X_test['y_true']
X_test = X_test.drop(columns=['y_true'])

X_train = df_transformed[(df_transformed['manufacturer_MODERNA'] == 0)]
y_train = X_train['y_true']
X_train = X_train.drop(columns=['y_true'])

In [79]:
X_train.to_csv('../Machine_Learning/X_train_Moderna.csv', index=False)
X_test.to_csv('../Machine_Learning/X_test_Moderna.csv', index=False)
y_train.to_csv('../Machine_Learning/y_train_Moderna.csv', index=False)
y_test.to_csv('../Machine_Learning/y_test_Moderna.csv', index=False)

# With Janssen as Test

In [82]:
X_test = df_transformed[(df_transformed['manufacturer_JANSSEN'] == 1)]
y_test = X_test['y_true']
X_test = X_test.drop(columns=['y_true'])

X_train = df_transformed[(df_transformed['manufacturer_JANSSEN'] == 0)]
y_train = X_train['y_true']
X_train = X_train.drop(columns=['y_true'])

In [84]:
X_train.to_csv('../Machine_Learning/X_train_Janssen.csv', index=False)
X_test.to_csv('../Machine_Learning/X_test_Janssen.csv', index=False)
y_train.to_csv('../Machine_Learning/y_train_Janssen.csv', index=False)
y_test.to_csv('../Machine_Learning/y_test_Janssen.csv', index=False)

# Create Embeddings for Dataset where a symptom only appears in test or train

### fist we want to know if there are any unique adverse reaction across the vainnes to split the dataset accoringly

In [102]:
Pfizer_adverse_reactions = [
    "Vomiting",
    "Lymphadenopathy",
    "Rash", 
    "Pruritus", 
    "Urticaria", 
    "Angioedema",
    "Anaphylaxis",
    "Decreased appetite",
    "Insomnia",
    "Headache",
    "Dizziness",
    "Lethargy",
    "Facial paralysis", 
    "Paraesthesia",
    "Hypoaesthesia",
    "Myocarditis",
    "Pericarditis",
    "Diarrhoea",
    "Nausea",
    "Hyperhidrosis",
    "Night sweats",
    "Erythema multiforme",
    "Arthralgia",
    "Myalgia",
    "Pain in extremity",
    "Heavy menstrual bleeding",
    "Injection site pain",
    "Fatigue",
    "Chills",
    "Pyrexia",
    "Injection site swelling",
    "Injection site redness",
    "Asthenia",
    "Malaise",
    "Injection site pruritus",
    "Extensive swelling of vaccinated limb",
    "Facial swelling"
]
Moderna_adverse_reactions = [
    "Lymphadenopathy",
    "Anaphylaxis",
    "Hypersensitivity",
    "Decreased appetite",
    "Irritability", 
    "Crying",
    "Headache",
    "Sleepiness",
    "Dizziness",
    "Facial paralysis", 
    "Hypoaesthesia",
    "Paraesthesia",
    "Myocarditis",
    "Pericarditis",
    "Nausea", 
    "Vomiting",
    "Diarrhoea",
    "Abdominal pain",
    "Rash",
    "Urticaria",
    "Erythema multiforme",
    "Mechanical urticaria",
    "Chronic urticaria",
    "Myalgia",
    "Arthralgia",
    "Heavy menstrual bleeding",
    "Injection site pain",
    "Fatigue",
    "Chills",
    "Pyrexia",
    "Injection site swelling",
    "Injection site erythema",
    "Injection site urticaria",
    "Injection site rash",
    "Delayed injection site reaction",
    "Injection site pruritus",
    "Facial swelling",
    "Extensive swelling of vaccinated limb"
]

Janssen_adverse_reactions = [
    "Lymphadenopathy",
    "Immune thrombocytopenia",
    "Anaphylaxis",
    "Headache",
    "Dizziness",
    "Tremor",
    "Urticaria",
    "Hypersensitivity",
    "Paraesthesia",
    "Hypoaesthesia",
    "Facial paralysis",
    "Tinnitus",
    "Guillain-Barre syndrome",
    "Myelitis", 
    "Venous thromboembolism",
    "Thrombosis in combination with thrombocytopenia",
    "Myocarditis", 
    "Pericarditis",
    "Capillary leak syndrome",
    "Cutaneous vasculitis", 
    "Nausea",
    "Cough", 
    "Oropharyngeal pain",
    "Sneezing",
    "Diarrhoea",
    "Vomiting",
    "Rash",
    "Hyperhidrosis",
    "Myalgia",
    "Arthralgia",
    "Muscular weakness",
    "Back pain",
    "Pain in extremity",
    "Fatigue",
    "Injection site pain",
    "Injection site swelling",
    "Chills",
    "Pyrexia"
]

Novavax_adverse_reactions = [
    "Headache",
    "Nausea",
    "Vomiting",
    "Myalgia",
    "Arthralgia",
    "Injection site tenderness",
    "Injection site pain",
    "Fatigue",
    "Malaise",
    "Injection site redness",
    "Injection site swelling",
    "Pyrexia",
    "Pain in extremity",
    "Injection site pruritus",
    "Chills",
    "Injection site warmth",
    "Lymphadenopathy",
    "Anaphylaxis",
    "Paraesthesia",
    "Hypoaesthesia",
    "Myocarditis",
    "Pericarditis",
    "Hypertension",
    "Rash",
    "Erythema",
    "Pruritus",
    "Urticaria"
]

In [107]:
from collections import Counter, defaultdict

named_lists = [
    ("Pfizer", Pfizer_adverse_reactions),
    ("Moderna", Moderna_adverse_reactions),
    ("Janssen", Janssen_adverse_reactions),
    ("Novavax", Novavax_adverse_reactions)
]

# Combine all lists into one
combined_list = []
for name, lst in named_lists:
    combined_list.extend(lst)

# Count the frequency of each word
word_counts = Counter(combined_list)

# Track the lists each word appears in
word_occurrences = defaultdict(set)

for name, lst in named_lists:
    for word in lst:
        word_occurrences[word].add(name)

# Print the word counts and the lists they appear in
for word, count in word_counts.items():
    lists_containing_word = ", ".join(word_occurrences[word])
    print(f"{word}: {count} times, appears in {lists_containing_word}")

Vomiting: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Lymphadenopathy: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Rash: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Pruritus: 2 times, appears in Pfizer, Novavax
Urticaria: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Angioedema: 1 times, appears in Pfizer
Anaphylaxis: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Decreased appetite: 2 times, appears in Pfizer, Moderna
Insomnia: 1 times, appears in Pfizer
Headache: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Dizziness: 3 times, appears in Janssen, Pfizer, Moderna
Lethargy: 1 times, appears in Pfizer
Facial paralysis: 3 times, appears in Janssen, Pfizer, Moderna
Paraesthesia: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Hypoaesthesia: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Myocarditis: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Pericarditis: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Dia

In [111]:
# Combine all lists into one
combined_list = []
for name, lst in named_lists:
    combined_list.extend(lst)

# Count the frequency of each word
word_counts = Counter(combined_list)

# Track the lists each word appears in
word_occurrences = defaultdict(set)

for name, lst in named_lists:
    for word in lst:
        word_occurrences[word].add(name)

# Print the word counts and the lists they appear in
print("Word counts and lists they appear in:")
for word, count in word_counts.items():
    lists_containing_word = ", ".join(word_occurrences[word])
    print(f"{word}: {count} times, appears in {lists_containing_word}")

# Function to print words that appear in a specific number of lists
def print_words_appearing_in_n_lists(n, lst_name, lst):
    words_in_n_lists = [word for word in lst if len(word_occurrences[word]) == n]
    words_and_lists = {word: word_occurrences[word] for word in words_in_n_lists}
    print(f"Words that appear in exactly {n} list(s): {len(words_in_n_lists)}")
    for word, lists in words_and_lists.items():
        print(f"  - {word}: appears in {', '.join(lists)}")

# Show per list how many words appear in exactly one, two, three, or four lists and which lists they appear in
for name, lst in named_lists:
    print(f"\n{name}:")
    print_words_appearing_in_n_lists(1, name, lst)
    print_words_appearing_in_n_lists(2, name, lst)
    print_words_appearing_in_n_lists(3, name, lst)
    print_words_appearing_in_n_lists(4, name, lst)

Word counts and lists they appear in:
Vomiting: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Lymphadenopathy: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Rash: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Pruritus: 2 times, appears in Pfizer, Novavax
Urticaria: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Angioedema: 1 times, appears in Pfizer
Anaphylaxis: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Decreased appetite: 2 times, appears in Pfizer, Moderna
Insomnia: 1 times, appears in Pfizer
Headache: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Dizziness: 3 times, appears in Janssen, Pfizer, Moderna
Lethargy: 1 times, appears in Pfizer
Facial paralysis: 3 times, appears in Janssen, Pfizer, Moderna
Paraesthesia: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Hypoaesthesia: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Myocarditis: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Pericarditis: 4 times, appears in

In [113]:
len(combined_list)

140

In [112]:
# Combine all lists into one
combined_list = []
for name, lst in named_lists:
    combined_list.extend(lst)

# Count the frequency of each word
word_counts = Counter(combined_list)

# Track the lists each word appears in
word_occurrences = defaultdict(set)

for name, lst in named_lists:
    for word in lst:
        word_occurrences[word].add(name)

# Print the word counts and the lists they appear in
print("Word counts and lists they appear in:")
for word, count in word_counts.items():
    lists_containing_word = ", ".join(word_occurrences[word])
    print(f"{word}: {count} times, appears in {lists_containing_word}")

# Show the total number of symptoms that appear one, two, three, and four times
appear_once = sum(1 for word in word_occurrences if len(word_occurrences[word]) == 1)
appear_twice = sum(1 for word in word_occurrences if len(word_occurrences[word]) == 2)
appear_thrice = sum(1 for word in word_occurrences if len(word_occurrences[word]) == 3)
appear_four_times = sum(1 for word in word_occurrences if len(word_occurrences[word]) == 4)

print("\nTotal number of symptoms appearing in lists:")
print(f"Appear in exactly one list: {appear_once}")
print(f"Appear in exactly two lists: {appear_twice}")
print(f"Appear in exactly three lists: {appear_thrice}")
print(f"Appear in exactly four lists: {appear_four_times}")

Word counts and lists they appear in:
Vomiting: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Lymphadenopathy: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Rash: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Pruritus: 2 times, appears in Pfizer, Novavax
Urticaria: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Angioedema: 1 times, appears in Pfizer
Anaphylaxis: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Decreased appetite: 2 times, appears in Pfizer, Moderna
Insomnia: 1 times, appears in Pfizer
Headache: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Dizziness: 3 times, appears in Janssen, Pfizer, Moderna
Lethargy: 1 times, appears in Pfizer
Facial paralysis: 3 times, appears in Janssen, Pfizer, Moderna
Paraesthesia: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Hypoaesthesia: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Myocarditis: 4 times, appears in Janssen, Pfizer, Moderna, Novavax
Pericarditis: 4 times, appears in

In [85]:
df = pd.read_csv('../Machine_Learning/df2_unique.csv')

In [86]:
df

,y_true,symptom,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,...,DE,De,dE,de,SERIOUS Y,SERIOUS N,manufacturer_JANSSEN,manufacturer_MODERNA,manufacturer_NOVAVAX,manufacturer_PFIZER\BIONTECH
0,1,Angioedema,23,95,161,304,308,287,191,105,...,2878.0,943022.0,2351.0,1530710.0,564.0,2314.0,0,0,0,1
1,1,Hypoaesthesia,39,732,2295,4248,4426,3566,1960,862,...,26529.0,919371.0,27845.0,1505216.0,7405.0,19126.0,0,0,0,1
2,1,Erythema multiforme,20,12,31,28,41,30,34,30,...,733.0,945167.0,2228.0,1530833.0,131.0,602.0,0,0,0,1
3,1,Insomnia,33,205,732,1345,1345,1253,1012,581,...,11215.0,934685.0,14609.0,1518452.0,2674.0,8541.0,0,0,0,1
4,1,Myalgia,106,935,4266,7133,6992,6599,4653,2335,...,52182.0,893718.0,75910.0,1457151.0,12832.0,39351.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,0,Hyperalbuminaemia,0,0,0,0,1,0,0,0,...,3.0,945897.0,0.0,1533061.0,1.0,2.0,0,0,0,1
1121,0,Injection site streaking,0,0,0,2,3,8,3,2,...,18.0,945882.0,525.0,1532536.0,1.0,17.0,0,0,0,1
1122,0,Retinopexy,0,0,0,1,1,0,3,1,...,8.0,945892.0,16.0,1533045.0,7.0,1.0,0,0,0,1
1123,0,Wound haemorrhage,0,1,5,4,8,12,3,7,...,79.0,945821.0,52.0,1533009.0,21.0,58.0,0,0,0,1


In [98]:
symptoms = df["symptom"]

In [99]:
symptoms

0                     Angioedema
1                  Hypoaesthesia
2            Erythema multiforme
3                       Insomnia
4                        Myalgia
                  ...           
1120           Hyperalbuminaemia
1121    Injection site streaking
1122                  Retinopexy
1123           Wound haemorrhage
1124               Cryoglobulins
Name: symptom, Length: 1125, dtype: object

In [87]:
X = df[['symptom']]
y = df['y_true']
print(X.shape)
print(y.shape)

(1125, 1)
(1125,)


,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99,...,symptom_embedding_40,symptom_embedding_41,symptom_embedding_42,symptom_embedding_43,symptom_embedding_44,symptom_embedding_45,symptom_embedding_46,symptom_embedding_47,symptom_embedding_48,symptom_embedding_49
0,23,95,161,304,308,287,191,105,45,7,...,-0.040381,0.020711,0.024002,0.007727,0.007641,-0.036815,-0.030769,0.037949,-0.032982,0.036309
1,39,732,2295,4248,4426,3566,1960,862,241,35,...,-0.010226,-0.003265,0.056631,-0.034765,-0.067561,0.009433,0.037754,0.012592,-0.031286,-0.056321
2,20,12,31,28,41,30,34,30,20,4,...,0.021166,0.068496,0.055908,-0.028799,-0.000966,-0.027396,0.087393,0.031979,0.000778,-0.049542
3,33,205,732,1345,1345,1253,1012,581,180,24,...,-0.020207,0.001275,0.026336,-0.048095,0.025708,0.041699,0.038123,-0.046873,0.041475,0.021545
4,106,935,4266,7133,6992,6599,4653,2335,583,76,...,0.029610,0.037658,-0.002482,-0.029510,-0.029124,-0.030801,-0.002990,0.060463,-0.000154,0.017201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,0,0,0,0,1,0,0,0,0,0,...,0.046631,-0.031411,-0.072253,0.014876,0.042463,0.056804,0.054448,0.013906,0.065797,0.007213
1121,0,0,0,2,3,8,3,2,0,0,...,-0.048126,0.026572,0.030746,-0.034195,-0.045404,0.014540,0.031676,-0.035743,0.029466,-0.025905
1122,0,0,0,1,1,0,3,1,0,0,...,-0.041297,0.048123,-0.004045,0.047992,0.025642,0.039317,-0.017408,-0.031193,0.030707,-0.033342
1123,0,1,5,4,8,12,3,7,1,2,...,0.013793,-0.047546,0.014219,0.031805,0.026704,-0.019001,-0.045651,0.004423,0.031366,0.041908


In [88]:
y.values

array([1, 1, 1, ..., 0, 0, 0])

In [89]:
embedding_info = ce.get_embedding_info(X)
X_encoded, encoders = ce.get_label_encoded_data(X)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y)

# Get embeddings
embeddings = ce.get_embeddings(X_train, y_train, categorical_embedding_info=embedding_info,
                               is_classification=True, epochs=100, batch_size=256)


Epoch 1/100
3/3 [==============================] - 1s 72ms/step - loss: 0.6695 - accuracy: 0.6424 - val_loss: 0.6093 - val_accuracy: 0.8639
Epoch 2/100
3/3 [==============================] - 0s 21ms/step - loss: 0.5706 - accuracy: 0.8917 - val_loss: 0.5141 - val_accuracy: 0.8639
Epoch 3/100
3/3 [==============================] - 0s 19ms/step - loss: 0.4557 - accuracy: 0.8917 - val_loss: 0.4241 - val_accuracy: 0.8639
Epoch 4/100
3/3 [==============================] - 0s 22ms/step - loss: 0.3569 - accuracy: 0.8917 - val_loss: 0.3835 - val_accuracy: 0.8639
Epoch 5/100
3/3 [==============================] - 0s 20ms/step - loss: 0.3066 - accuracy: 0.8917 - val_loss: 0.3966 - val_accuracy: 0.8639
Epoch 6/100
3/3 [==============================] - 0s 20ms/step - loss: 0.2981 - accuracy: 0.8917 - val_loss: 0.3965 - val_accuracy: 0.8639
Epoch 7/100
3/3 [==============================] - 0s 25ms/step - loss: 0.2752 - accuracy: 0.8917 - val_loss: 0.3640 - val_accuracy: 0.8639
Epoch 8/100
3/3 [===

In [90]:
df_embedded = ce.fit_transform(df, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
df_embedded.head()

,y_true,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,...,symptom_embedding_40,symptom_embedding_41,symptom_embedding_42,symptom_embedding_43,symptom_embedding_44,symptom_embedding_45,symptom_embedding_46,symptom_embedding_47,symptom_embedding_48,symptom_embedding_49
0,1,23,95,161,304,308,287,191,105,45,...,-0.040381,0.020711,0.024002,0.007727,0.007641,-0.036815,-0.030769,0.037949,-0.032982,0.036309
1,1,39,732,2295,4248,4426,3566,1960,862,241,...,-0.010226,-0.003265,0.056631,-0.034765,-0.067561,0.009433,0.037754,0.012592,-0.031286,-0.056321
2,1,20,12,31,28,41,30,34,30,20,...,0.021166,0.068496,0.055908,-0.028799,-0.000966,-0.027396,0.087393,0.031979,0.000778,-0.049542
3,1,33,205,732,1345,1345,1253,1012,581,180,...,-0.020207,0.001275,0.026336,-0.048095,0.025708,0.041699,0.038123,-0.046873,0.041475,0.021545
4,1,106,935,4266,7133,6992,6599,4653,2335,583,...,0.029610,0.037658,-0.002482,-0.029510,-0.029124,-0.030801,-0.002990,0.060463,-0.000154,0.017201


In [91]:
num_attribs = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99', '100-109',
               '110-119', 'NUMDAYS_0-1', 'NUMDAYS_2-3', 'NUMDAYS_4-5', 'NUMDAYS_6-7', 'NUMDAYS_8-9', 'NUMDAYS_10-11',
               'NUMDAYS_12-13', 'NUMDAYS_14+', 'DIED', 'L_THREAT', 'HOSPITAL', 'X_STAY', 'DISABLE', 'BIRTH_DEFECT',
               'DE', 'De', 'dE', 'de', 'SERIOUS Y', 'SERIOUS N', 'SEX_M', 'SEX_F', 'SEX_U']

# Separate features and target
X = df_embedded.drop(columns=['y_true'])
y_true = df_embedded['y_true']

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('min_max_scaler', MinMaxScaler())
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs)
], remainder='passthrough')

# Apply transformation
X_transformed = full_pipeline.fit_transform(X)

# Combine the transformed features with the target
df_transformed = pd.DataFrame(X_transformed, columns=X.columns)

In [93]:
df_transformed.columns

Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80-89', '90-99', '100-109', '110-119', 'NUMDAYS_0-1', 'NUMDAYS_2-3',
       'NUMDAYS_4-5', 'NUMDAYS_6-7', 'NUMDAYS_8-9', 'NUMDAYS_10-11',
       'NUMDAYS_12-13', 'NUMDAYS_14+', 'SEX_M', 'SEX_F', 'SEX_U', 'DIED',
       'L_THREAT', 'HOSPITAL', 'X_STAY', 'DISABLE', 'BIRTH_DEFECT', 'DE', 'De',
       'dE', 'de', 'SERIOUS Y', 'SERIOUS N', 'manufacturer_JANSSEN',
       'manufacturer_MODERNA', 'manufacturer_NOVAVAX',
       'manufacturer_PFIZER\BIONTECH', 'symptom_embedding_0',
       'symptom_embedding_1', 'symptom_embedding_2', 'symptom_embedding_3',
       'symptom_embedding_4', 'symptom_embedding_5', 'symptom_embedding_6',
       'symptom_embedding_7', 'symptom_embedding_8', 'symptom_embedding_9',
       'symptom_embedding_10', 'symptom_embedding_11', 'symptom_embedding_12',
       'symptom_embedding_13', 'symptom_embedding_14', 'symptom_embedding_15',
       'symptom_embedding_16', 'symptom_embedding_1

In [94]:
df_transformed

,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99,...,symptom_embedding_40,symptom_embedding_41,symptom_embedding_42,symptom_embedding_43,symptom_embedding_44,symptom_embedding_45,symptom_embedding_46,symptom_embedding_47,symptom_embedding_48,symptom_embedding_49
0,0.020193,0.016728,0.014843,0.017429,0.017247,0.016329,0.012564,0.011758,0.008757,0.004587,...,-0.040381,0.020711,0.024002,0.007727,0.007641,-0.036815,-0.030769,0.037949,-0.032982,0.036309
1,0.034241,0.128896,0.211579,0.243550,0.247844,0.202890,0.128930,0.096529,0.046896,0.022936,...,-0.010226,-0.003265,0.056631,-0.034765,-0.067561,0.009433,0.037754,0.012592,-0.031286,-0.056321
2,0.017559,0.002113,0.002858,0.001605,0.002296,0.001707,0.002237,0.003359,0.003892,0.002621,...,0.021166,0.068496,0.055908,-0.028799,-0.000966,-0.027396,0.087393,0.031979,0.000778,-0.049542
3,0.028973,0.036098,0.067484,0.077113,0.075316,0.071290,0.066570,0.065062,0.035026,0.015727,...,-0.020207,0.001275,0.026336,-0.048095,0.025708,0.041699,0.038123,-0.046873,0.041475,0.021545
4,0.093064,0.164642,0.393288,0.408955,0.391533,0.375455,0.306078,0.261478,0.113446,0.049803,...,0.029610,0.037658,-0.002482,-0.029510,-0.029124,-0.030801,-0.002990,0.060463,-0.000154,0.017201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,0.000000,0.000000,0.000000,0.000000,0.000056,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.046631,-0.031411,-0.072253,0.014876,0.042463,0.056804,0.054448,0.013906,0.065797,0.007213
1121,0.000000,0.000000,0.000000,0.000115,0.000168,0.000455,0.000197,0.000224,0.000000,0.000000,...,-0.048126,0.026572,0.030746,-0.034195,-0.045404,0.014540,0.031676,-0.035743,0.029466,-0.025905
1122,0.000000,0.000000,0.000000,0.000057,0.000056,0.000000,0.000197,0.000112,0.000000,0.000000,...,-0.041297,0.048123,-0.004045,0.047992,0.025642,0.039317,-0.017408,-0.031193,0.030707,-0.033342
1123,0.000000,0.000176,0.000461,0.000229,0.000448,0.000683,0.000197,0.000784,0.000195,0.001311,...,0.013793,-0.047546,0.014219,0.031805,0.026704,-0.019001,-0.045651,0.004423,0.031366,0.041908


In [100]:
df_transformed["sypmtom"] = symptoms

In [101]:
df_transformed

,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99,...,symptom_embedding_41,symptom_embedding_42,symptom_embedding_43,symptom_embedding_44,symptom_embedding_45,symptom_embedding_46,symptom_embedding_47,symptom_embedding_48,symptom_embedding_49,sypmtom
0,0.020193,0.016728,0.014843,0.017429,0.017247,0.016329,0.012564,0.011758,0.008757,0.004587,...,0.020711,0.024002,0.007727,0.007641,-0.036815,-0.030769,0.037949,-0.032982,0.036309,Angioedema
1,0.034241,0.128896,0.211579,0.243550,0.247844,0.202890,0.128930,0.096529,0.046896,0.022936,...,-0.003265,0.056631,-0.034765,-0.067561,0.009433,0.037754,0.012592,-0.031286,-0.056321,Hypoaesthesia
2,0.017559,0.002113,0.002858,0.001605,0.002296,0.001707,0.002237,0.003359,0.003892,0.002621,...,0.068496,0.055908,-0.028799,-0.000966,-0.027396,0.087393,0.031979,0.000778,-0.049542,Erythema multiforme
3,0.028973,0.036098,0.067484,0.077113,0.075316,0.071290,0.066570,0.065062,0.035026,0.015727,...,0.001275,0.026336,-0.048095,0.025708,0.041699,0.038123,-0.046873,0.041475,0.021545,Insomnia
4,0.093064,0.164642,0.393288,0.408955,0.391533,0.375455,0.306078,0.261478,0.113446,0.049803,...,0.037658,-0.002482,-0.029510,-0.029124,-0.030801,-0.002990,0.060463,-0.000154,0.017201,Myalgia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,0.000000,0.000000,0.000000,0.000000,0.000056,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.031411,-0.072253,0.014876,0.042463,0.056804,0.054448,0.013906,0.065797,0.007213,Hyperalbuminaemia
1121,0.000000,0.000000,0.000000,0.000115,0.000168,0.000455,0.000197,0.000224,0.000000,0.000000,...,0.026572,0.030746,-0.034195,-0.045404,0.014540,0.031676,-0.035743,0.029466,-0.025905,Injection site streaking
1122,0.000000,0.000000,0.000000,0.000057,0.000056,0.000000,0.000197,0.000112,0.000000,0.000000,...,0.048123,-0.004045,0.047992,0.025642,0.039317,-0.017408,-0.031193,0.030707,-0.033342,Retinopexy
1123,0.000000,0.000176,0.000461,0.000229,0.000448,0.000683,0.000197,0.000784,0.000195,0.001311,...,-0.047546,0.014219,0.031805,0.026704,-0.019001,-0.045651,0.004423,0.031366,0.041908,Wound haemorrhage


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_transformed, y_true, test_size=0.2, random_state=42)

X_train.to_csv('../Machine_Learning/X_train_unique.csv', index=False)
X_test.to_csv('../Machine_Learning/X_test_unique.csv', index=False)
y_train.to_csv('../Machine_Learning/y_train_unique.csv', index=False)
y_test.to_csv('../Machine_Learning/y_test_unique.csv', index=False)